In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from math import sin
import pandas as pd
import os
import numpy as np

In [15]:
train = True
accountDB = pd.read_csv('data/account.csv', sep=";")
cardDB = pd.read_csv('data/card_train.csv', sep=";") if train else pd.read_csv('data/card_test.csv', sep=";")
dispositionDB = pd.read_csv('data/disp.csv', sep=";")
districtDB = pd.read_csv('data/district.csv', sep=";")
loanDB = pd.read_csv('data/loan_train.csv', sep=";") if train else pd.read_csv('data/loan_test.csv', sep=";") 
transactionDB = pd.read_csv('data/trans_train.csv', sep=";")
transactions_ready = pd.read_csv('data/trans_train_ready.csv') if train else pd.read_csv('data/trans_test_ready.csv')

/home/iohan-sardinha/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3444: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [16]:
diff = (districtDB.drop(index=68)["unemploymant rate '96 "] -  districtDB.drop(index=68)["unemploymant rate '95 "].astype(float)).to_frame()
diff = diff.drop(index=70)
mean_ = diff[0].median()
#plt.axhline(y=mean_, label="Avarage", linewidth=1, color="orange")
#plt.plot(diff.index, diff[0])

In [17]:
districtDB.at[68,"unemploymant rate '95 "] = districtDB.at[68,"unemploymant rate '96 "] - mean_
districtDB["unemploymant rate '95 "] = districtDB.at[68,"unemploymant rate '95 "].astype(float)

districtDB.at[68,"no. of commited crimes '95 "] = districtDB.at[68,"no. of commited crimes '96 "]
districtDB["no. of commited crimes '95 "] = districtDB.at[68,"no. of commited crimes '95 "].astype(float)

In [18]:
clientDB = pd.read_csv('data/client.csv', sep=";")
clientDB.insert(0, "women",(((clientDB['birth_number'] - ((clientDB['birth_number']//10000 * 10000) + (clientDB['birth_number'] - clientDB['birth_number']//100 * 100)))//100) > 50))
clientDB.insert(0,"age", 1998 - (1900 + clientDB['birth_number']//10000))
clientDB["birthday"] = np.where(clientDB["women"], clientDB['birth_number']-5000, clientDB['birth_number'])
clientDB = clientDB.drop(columns="birth_number")

In [19]:
loanDB = loanDB.rename(columns={"amount":"loan_value","date":"loan_date", "duration":"loan_duration", "status": "loan_success"})
dispositionDB = dispositionDB.rename(columns={"type":"disposition_type"})
accountDB = accountDB.rename(columns={"date":"account_creation", "frequency":"account_frequency"})

In [20]:
clientWthDistrict = clientDB.merge(districtDB, left_on="district_id", right_on="code ")
clientWthDistrict = clientWthDistrict.drop(columns=["name ", "district_id"]) 
clientWthDistrict =  clientWthDistrict.rename(columns={col:("client_district "+col) for col in clientWthDistrict.columns[4:]})

In [21]:
accountWthDistrict = accountDB.merge(districtDB, left_on="district_id", right_on="code ")
accountWthDistrict = accountWthDistrict.drop(columns=["name ", "district_id"])
accountWthDistrict =  accountWthDistrict.rename(columns={col:("account_district "+col) for col in accountWthDistrict.columns[3:]})

In [22]:
transactions_ready = transactions_ready.rename(columns={"amount":"no. of transactions"})

In [23]:
cardDB = cardDB.rename(columns={"type":"card_type", "issued":"card issue"})

In [24]:
cardWithClient = cardDB.merge(dispositionDB).drop(columns=["card_id","disp_id","client_id","disposition_type"])
loanClientsWithNoCard = [i for i in loanDB["account_id"] if not i in list(cardWithClient["account_id"])]
cardClientsWithNoLoan = [i for i in cardWithClient["account_id"] if not i in list(loanDB["account_id"])]
cardWithClient = cardWithClient.drop(index=cardWithClient[cardWithClient["account_id"].isin(cardClientsWithNoLoan)].index)
loanClientsWithNoCard = np.array([loanClientsWithNoCard,["no card"]*len(loanClientsWithNoCard),[0]*len(loanClientsWithNoCard)]).transpose()
loanClientsWithNoCard = pd.DataFrame(loanClientsWithNoCard,columns=["account_id","card_type", "card issue"])
cardWithClient = cardWithClient.append(loanClientsWithNoCard)
cardWithClient["account_id"] = cardWithClient["account_id"].astype(int)

In [25]:
accountClient = dispositionDB.merge(accountWthDistrict)
accountClient = accountClient.merge(clientWthDistrict)
accountClient = accountClient.drop(columns="client_id")
accountClient = accountClient[accountClient["disposition_type"] == "OWNER"]

In [26]:
loanFinal = loanDB.merge(accountClient, on="account_id")
loanFinal = loanFinal.merge(cardWithClient, on="account_id")
loanFinal = loanFinal.merge(transactions_ready, on="account_id")
loanFinal = loanFinal.drop(columns=["account_id", "disp_id", "disposition_type"])

In [27]:
loanFinal.to_csv("data/loanUnited"+("Train" if train else "Test") + ".csv", index=False)
loanFinal

,loan_id,loan_date,loan_value,loan_duration,payments,loan_success,account_frequency,account_creation,account_district code,account_district region,...,card issue,no. of transactions,balance,nr_movements,min_trans_amount,max_trans_amount,avg_trans_amount,min_balance,max_balance,avg_balance
0,5314,930705,96396,12,8033,-1,weekly issuance,930322,30,west Bohemia,...,0,3300.0,20100.0,4.0,1100.0,9900.0,5025.000000,1100.0,20100.0,12250.000000
1,5316,930711,165960,36,4610,1,monthly issuance,930213,46,east Bohemia,...,0,3419.0,52208.9,37.0,2.9,54300.0,11015.635135,700.0,120512.8,52083.859459
2,6863,930728,127080,60,2118,1,monthly issuance,930208,45,east Bohemia,...,0,12000.0,20272.8,24.0,48.6,19065.0,5417.458333,800.0,49590.4,30060.954167
3,5325,930803,105804,36,2939,1,monthly issuance,930130,12,central Bohemia,...,0,14.6,34292.7,25.0,14.6,26448.0,8253.080000,1000.0,65898.5,41297.480000
4,7240,930906,274740,60,4579,1,weekly issuance,930214,1,Prague,...,0,182.8,41142.9,27.0,30.0,63366.0,18945.966667,600.0,122893.1,57188.211111
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,6818,961212,155616,48,3242,1,monthly issuance,950121,72,north Moravia,...,0,14600.0,60694.1,172.0,14.6,36960.0,4895.974419,200.0,75294.1,44197.509884
324,5625,961215,222180,60,3703,-1,monthly issuance,951129,29,west Bohemia,...,0,6900.0,59578.8,59.0,14.6,52600.0,11725.494915,800.0,130659.1,55230.444068
325,6805,961221,45024,48,938,1,monthly issuance,960521,70,north Moravia,...,0,17800.0,38384.3,39.0,14.6,31636.5,7779.815385,800.0,63659.3,41994.907692
326,7233,961225,115812,36,3217,1,monthly issuance,950520,16,south Bohemia,...,0,3100.0,41878.1,124.0,14.6,50800.0,13280.084677,1100.0,119527.2,56646.516129
